#### Testing Local Sagemaker SDK

In [ ]:
import sagemaker
from sagemaker.local import LocalSession
from sagemaker.model import Model
from sagemaker.predictor import Predictor
import json, cv2

DUMMY_IAM_ROLE = 'arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-20200101T000001'

YOUR_BUCKET_PREFIX = 's3://<your BUCKET>/<your prefixes>'

In [ ]:
# It's locally but aws SDK needs a default profile setup 
# If you already have this, you can safelly ignore this step
%env AWS_DEFAULT_REGION=<your region>
%env AWS_DEFAULT_PROFILE=<your profile>

In [ ]:
sagemaker_session = LocalSession()
sagemaker_session.config = {'local': {'local_code': True}}

In [ ]:
### It 's necessary to package model into a tar.gz
!rm model.tar.gz
!cd model && tar -cvzf ../model.tar.gz Detic_LCOCOI21k_CLIP_SwinB_896b32_4x_ft4x_max-size.pth && cd ..

In [ ]:
## Upload to S3
local_model = 'model.tar.gz'
s3uri = sagemaker.s3.S3Uploader.upload(local_model, YOUR_BUCKET_PREFIX)
s3uri

In [ ]:
role = DUMMY_IAM_ROLE
model_dir = s3uri
image = '<your image hosted on ECR repository>'

In [ ]:
def predict_wrapper(endpoint, session):
    return Predictor(endpoint, session)

model = Model(
    image_uri=image,
    role=role,
    model_data=model_dir,
    sagemaker_session=sagemaker_session,
    predictor_cls=predict_wrapper
)

In [ ]:
from sagemaker.predictor import json_serializer

print('Deploying endpoint in local mode')
print('Note: if launching for the first time in local mode, container image download might take a few minutes to complete.')
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local',
    serializer=json_serializer
)

In [ ]:
print(predictor)

In [ ]:
from matplotlib import pyplot as plt

import cv2
#import requests

from PIL import Image
import json
import base64
from io import BytesIO

In [ ]:
im = cv2.imread("./desk.jpg")
plt.figure(figsize = (60,20))
plt.imshow(im)
plt.show()

In [ ]:
# Prepare Image to send
img = Image.open("desk.jpg")

#Convert Pillow Image to bytes and then to base64
buffered = BytesIO()
img.save(buffered, format="JPEG")
img_byte = buffered.getvalue() # bytes
img_base64 = base64.b64encode(img_byte) #Base64-encoded bytes * not str

#It's still bytes so json.Convert to str to dumps(Because the json element does not support bytes type)
img_str = img_base64.decode('utf-8') # str

files = json.dumps({
    "labels":"cup,mouse", #change to labels that will be detected
    "img":img_str
})

In [ ]:
result = predictor.predict(files)

In [ ]:
# Detectron2 to load response
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import jsonpickle

metadata = MetadataCatalog.get("__unused")

In [ ]:

frame = jsonpickle.decode(result)

v = Visualizer(im[:, :, ::-1], metadata)
out = v.draw_instance_predictions(frame["instances"].to("cpu"))
plt.figure(figsize = (60,20))
plt.imshow(out.get_image())
plt.show()

In [ ]:
## Deleting 
predictor.delete_endpoint()